In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

from tqdm.notebook import tqdm
import pickle
import os
import numpy as np
from numpy.linalg import norm
import shutil
import time

from get_less_acc_classes import get_classes

In [2]:
# Загрузка класс-идентификаторов
with open('./data/class_ids-caltech101.pickle', 'rb') as f:
    class_ids = pickle.load(f)

# Загрузка списка имен файлов
with open('./data/filenames-caltech101.pickle', 'rb') as f:
    filenames = pickle.load(f)

# Загрузка списка признаков
with open('./data/features-caltech101-' + 'resnet50' + '.pickle', 'rb') as f:
    features = pickle.load(f)


In [3]:
TRAIN_SAMPLES = 8677
NUM_CLASSES = 101
BATCH_SIZE = 128
NUM_EPOCHS = 30
PATIENCE = 5  # Количество эпох для ранней остановки
IMG_WIDTH, IMG_HEIGHT = 224, 224
ROOT_DIR = './datasets/caltech101'
base_model = ResNet50(weights='imagenet',
                         include_top=False,
                         input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

In [4]:
less_accurate_classnames = get_classes(
ROOT_DIR,
'data/filenames-caltech101.pickle',
'data/features-caltech101-resnet50.pickle',
'data/class_ids-caltech101.pickle'
)

c:\my\DL\CV\chapter-2\.conda\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Общая точность классификации: 91.53%
	bonsai: 97.56%
	Faces_easy: 96.70%
	brain: 95.45%
	butterfly: 95.24%
	ewer: 95.24%
	cougar_face: 94.74%
	grand_piano: 94.44%
	llama: 94.12%
	nautilus: 94.12%
	cup: 93.75%
	elephant: 93.75%
	lamp: 93.33%
	lotus: 93.33%
	tick: 92.31%
	chair: 91.67%
	sea_horse: 91.67%
	crocodile_head: 90.91%
	cougar_body: 90.00%
	dolphin: 90.00%
	emu: 90.00%
	flamingo: 88.89%
	metronome: 88.89%
	helicopter: 88.24%
	yin_yang: 87.50%
	electric_guitar: 86.67%
	windsor_chair: 86.67%
	inline_skate: 85.71%
	scissors: 85.71%
	dollar_bill: 84.62%
	stapler: 84.62%
	gerenuk: 83.33%
	pyramid: 83.33%
	snoopy: 83.33%
	strawberry: 83.33%
	stegosaurus: 78.57%
	umbrella: 77.78%
	cannon: 75.00%
	okapi: 75.00%
	gramophone: 73.33%
	crayfish: 72.73%
	beaver: 66.67%
	ceiling_fan: 62.50%
	flamingo_head: 62.50%
	mayfly: 62.50%
	schooner: 62.50%
	ant: 60.00%
	barrel: 57.14%
	bass: 54.55%
	brontosaurus: 50.00%
	wrench: 44.44%
	octopus: 42.86%
	crocodile: 37.50%
	lobster: 33.33%
	wheelchair: 3

In [5]:
if os.path.exists('augmented_data'):
    shutil.rmtree('augmented_data')

# Копируем содержимое папки
shutil.copytree(ROOT_DIR, 'augmented_data')
print(f"Копирование папки '{ROOT_DIR}' в augmented_data успешно завершено.")

Копирование папки './datasets/caltech101' в augmented_data успешно завершено.


In [6]:
def get_images_increment(accuracy):
    if 70 <= accuracy < 100:
        return 2
    elif 50 <= accuracy < 70:
        return 3
    elif 30 <= accuracy < 50:
        return 4
    elif 5 <= accuracy < 30:
        return 5
    else:
        return 1  # По умолчанию генерировать 1 копию

for class_name, accuracy in less_accurate_classnames.items():
    images_increment = get_images_increment(accuracy)

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    class_dir = os.path.join(ROOT_DIR, class_name)
    class_images = os.listdir(class_dir)

    for img_name in class_images:
        img_path = os.path.join(class_dir, img_name)
        img = tf.keras.preprocessing.image.load_img(img_path)
        x = tf.keras.preprocessing.image.img_to_array(img)
        x = np.expand_dims(x, axis=0)

        i = 0

        
        for batch in datagen.flow(x, batch_size=1, save_to_dir=os.path.join('augmented_data', class_name), save_prefix=class_name, save_format='jpg'):
            i += 1
            if i >= images_increment:  # Аугментация n копий для каждого изображения
                break


In [7]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   validation_split=0.2  # 20% данных для валидации
)

In [8]:
train_generator = train_datagen.flow_from_directory(
    'augmented_data',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=12345,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'augmented_data',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical',
    subset='validation'
)

Found 13551 images belonging to 101 classes.
Found 3331 images belonging to 101 classes.


In [9]:
tf.keras.mixed_precision.experimental.set_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060 Laptop GPU, compute capability 8.6


In [10]:
def model_maker():
    base_model = ResNet50(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.2)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [11]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [12]:
import multiprocessing
num_cores = multiprocessing.cpu_count()
print(f"Number of available CPU cores: {num_cores}")

Number of available CPU cores: 12


In [13]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=PATIENCE),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_acc', save_best_only=True)
]

model.fit(train_generator, 
          epochs=NUM_EPOCHS, 
          validation_data=validation_generator, 
          validation_steps=len(validation_generator), 
          steps_per_epoch=len(train_generator), 
          verbose=1,
          workers=num_cores,
          callbacks=callbacks)

Epoch 1/30
106/106 [==============================] - 40s 285ms/step - loss: 2.4853 - acc: 0.4542 - val_loss: 1.0487 - val_acc: 0.7655
Epoch 2/30
106/106 [==============================] - 35s 306ms/step - loss: 0.9963 - acc: 0.7514 - val_loss: 0.5509 - val_acc: 0.8718
Epoch 3/30
106/106 [==============================] - 35s 310ms/step - loss: 0.6875 - acc: 0.8154 - val_loss: 0.4003 - val_acc: 0.9030
Epoch 4/30
106/106 [==============================] - 35s 305ms/step - loss: 0.5313 - acc: 0.8526 - val_loss: 0.3547 - val_acc: 0.9024
Epoch 5/30
106/106 [==============================] - 35s 307ms/step - loss: 0.4495 - acc: 0.8711 - val_loss: 0.3189 - val_acc: 0.9114
Epoch 6/30
106/106 [==============================] - 35s 308ms/step - loss: 0.3958 - acc: 0.8861 - val_loss: 0.2734 - val_acc: 0.9270
Epoch 7/30
106/106 [==============================] - 88s 816ms/step - loss: 0.3574 - acc: 0.8963 - val_loss: 0.2468 - val_acc: 0.9289
Epoch 8/30
106/106 [==============================] - 3

In [14]:
TUNED_MODEL = load_model('best_model.h5')

In [15]:
# Извлекаю признаки
datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

generator = datagen.flow_from_directory('augmented_data',
                                        target_size=(224, 224),
                                        class_mode=None,
                                        shuffle=False)

start_time = time.time()
feature_list = []
feature_list = TUNED_MODEL.predict(generator, NUM_EPOCHS)
end_time = time.time()

Found 16882 images belonging to 101 classes.


In [16]:
for i, features in enumerate(feature_list):
    feature_list[i] = features / norm(features)

feature_list = feature_list.reshape(len(feature_list), -1)

print("Num images   = ", len(generator.classes))
print("Shape of feature_list = ", feature_list.shape)
print("Time taken in sec = ", end_time - start_time)

Num images   =  16882
Shape of feature_list =  (16882, 101)
Time taken in sec =  29.39704179763794


In [17]:
filenames = ['augmented_data' + '/' + s for s in generator.filenames]

In [18]:
pickle.dump(generator.classes, open('./data/tuned_class_ids-caltech101.pickle',
                                    'wb'))
pickle.dump(filenames, open('./data/tuned_filenames-caltech101.pickle', 'wb'))
pickle.dump(
    feature_list,
    open('./data/tuned_features-caltech101-' + TUNED_MODEL.name + '.pickle', 'wb'))

In [19]:
# Сравнение
new_list = get_classes(
'augmented_data',
'./data/tuned_filenames-caltech101.pickle',
'./data/tuned_features-caltech101-' + TUNED_MODEL.name + '.pickle',
'./data/tuned_class_ids-caltech101.pickle',
)

c:\my\DL\CV\chapter-2\.conda\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Общая точность классификации: 99.26%
	airplanes: 98.73%
	butterfly: 98.31%
	electric_guitar: 97.78%
	crayfish: 97.67%
	lotus: 97.14%
	cannon: 96.77%
	ketch: 96.43%
	schooner: 96.15%
	sunflower: 95.00%
	cougar_face: 94.87%
	buddha: 94.74%
	menorah: 94.74%
	anchor: 94.44%
	dragonfly: 94.44%
	kangaroo: 94.12%
	crab: 92.86%
	crocodile_head: 91.30%
	pagoda: 90.00%
	headphone: 87.50%
	camera: 80.00%
	garfield: 80.00%
